In [1]:
import numpy as np
import json
import pandas as pd
import os
import re
import ast
import xml.etree.ElementTree as ET
import ast

from __future__ import print_function
from gensim import corpora

from nltk.sem.logic import *
from nltk.sem.logic import LogicParser

In [2]:
pre_var = []
formulas = []
check_dup = set([])

from nltk.sem.logic import LogicParser
from nltk.sem.logic import *

logic_parser = LogicParser(type_check=False)
def lexpr(formula_str):
    return logic_parser.parse(formula_str)

def coq_string_expr(expression):
    
    org_exp = expression
    
    if isinstance(expression, str):
        expression = lexpr(expression)
    expr_coq_str = ''
    if isinstance(expression, ApplicationExpression):
        coq_string_application_expr(expression)
    elif isinstance(expression, AbstractVariableExpression):
        expr_coq_str = coq_string_abstract_variable_expr(expression)
    elif isinstance(expression, LambdaExpression):
        raise
    elif isinstance(expression, QuantifiedExpression):
        expr_coq_str = coq_string_quantified_expr(expression)
    elif isinstance(expression, AndExpression):
        expr_coq_str = coq_string_and_expr(expression)
    elif isinstance(expression, OrExpression):
        expr_coq_str = coq_string_or_expr(expression)
    elif isinstance(expression, NegatedExpression):
        expr_coq_str = coq_string_not_expr(expression)
    elif isinstance(expression, BinaryExpression):
        expr_coq_str = coq_string_binary_expr(expression)
    elif isinstance(expression, Variable):
        expr_coq_str = '%s' % expression
    else:
        expr_coq_str = str(expression)

    return org_exp

coqstr = coq_string_expr

def coq_string_application_expr(expression):
    # uncurry the arguments and find the base function
    if expression.is_atom():
        function, args = expression.uncurry()
        arg_str = ' '.join("%s" % coqstr(arg) for arg in args)
    else:
        #Leave arguments curried
        function = expression.function
        arg_str = "%s" % coqstr(expression.argument)
    function_str = "%s" % coqstr(function)
    parenthesize_function = False
    if isinstance(function, LambdaExpression):
        if isinstance(function.term, ApplicationExpression):
            if not isinstance(function.term.function,
                              AbstractVariableExpression):
                parenthesize_function = True
        elif not isinstance(function.term, BooleanExpression):
            parenthesize_function = True
    elif isinstance(function, ApplicationExpression):
        parenthesize_function = True
    if parenthesize_function:
        function_str = Tokens.OPEN + function_str + Tokens.CLOSE
    return expression 

def coq_string_abstract_variable_expr(expression):
    expr_str = str(expression.variable)
    if not isinstance(expression, FunctionVariableExpression):
        if expr_str == '':
            expr_str = "%s" % expr_str
        else:
            expr_str = "%s" % expr_str
    else:
        expr_str = "%s" % expr_str
        
    if not re.sub(r'_', "",expr_str) in check_dup:
        if not expr_str.startswith('_'):
            pre_var.append(expr_str)
            check_dup.add(expr_str)
    return expression


def coq_string_quantified_expr(expression):
    variables = [expression.variable]
    term = expression.term
    while term.__class__ == expression.__class__:
        variables.append(term.variable)
        term = term.term
    for v in variables:
        coqstr(v)
    coqstr(term) 
    return expression

def coq_string_and_expr(expression):
    first = coqstr(expression.first)
    second = coqstr(expression.second)
    return expression 

def coq_string_or_expr(expression):
    first = coqstr(expression.first)
    second = coqstr(expression.second)
    return expression

def coq_string_not_expr(expression):
    term_str = coqstr(expression.term)
    return expression

def coq_string_binary_expr(expression):
    first = coqstr(expression.first)
    second = coqstr(expression.second)
    return expression


In [3]:
def substituteString(text,lst):
    ###辞書作り###
    lst = list(set(lst))
    e_sub = {}
    z_sub = {}
    ice = 0 #index counter
    icz = 0 

    for i in lst:
        matchObj = re.search(r'e0+', i)
        if(matchObj):
            tmp = "e0"+str(ice)
            e_sub[tmp] = i
            ice+=1
            continue
        matchObj = re.search(r'z[0-9]*', i)
        if matchObj:
            tmp = "z0"+str(icz)
            z_sub[tmp] = i
            icz+=1
    
    e_sub = {v:k for k, v in e_sub.items()}
    z_sub = {v:k for k, v in z_sub.items()}
    
    for k, v in e_sub.items():  
        text = text.replace(k,v)
    for k, v in z_sub.items():  
        text = text.replace(k,v)
    return text

In [4]:
tree = ET.parse('sem/snli_dev.txt.candc.sem.xml')
root = tree.getroot()

root = root[0]
root = root[0] #１個目のsentence

mydict = {}
#c = 0 #辞書のindexを回す
print(len(root))

#子階層のタグと中身
for c,child in enumerate(root):
    
    formula = child[2]  #child[2]がsemantics
    check = (child[2].attrib)
    check = check['status']
   
    if(check == 'success'):
        plain = ""
        toridashi = child[0]
        
        for i in toridashi :
            p = i.attrib
            p = p['surf']
            if(p=='.' or p== ','):
                plain = plain + p
            else :
                plain = plain + " " + p
        plain = plain+'\n'
        
        formula = child[2][0]
        formula = (formula.attrib)
        formula = formula['sem'] #\nが付与されていると信じている
        
        #論理式を今回の標準にする
        check_dup = set([])
        pre_var = []
        
        try:
            org_f = formula
            formula = coq_string_expr(formula)
        except:
            #import traceback
            #traceback.print_exc()
            continue
            
        if(formula == 'lambda_expression'):
            continue
        formula = substituteString(formula,pre_var)
        ice = 0
        icz = 0

        for i in range(len(pre_var)):
            matchObj = re.search(r'e0+', pre_var[i])
            if(matchObj):
                tmp = "e0"+str(ice)
                pre_var[i] = tmp

            matchObj = re.search(r'z[0-9]*', pre_var[i])
            if matchObj:
                tmp = "z0"+str(icz)
                pre_var[i] = tmp
                icz+=1

        pair = {'text':plain,'formula':formula}
        mydict.update({str(c):pair})
            
    else:
      print('faild parse')
    #c+=1
    #if(c==100):
    #    break
print(len(mydict))
    
f = open('snli_input_data_100_1214.json', 'w') # 書き込みモードで開く
json.dump(mydict, f,ensure_ascii=False)
f.close()

20000


IndexError: child index out of range

In [8]:
print(len(mydict))

100
